In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from preprocess import pack
from features import Features

feat=Features(**pack)
data=feat.execute()
data

,season,datetime,ftr,htr,referee,hometeam,awayteam,ftg_hr,htg_hr,s_hr,...,ftg_car,htg_car,s_car,st_car,c_car,f_car,y_car,r_car,p_car,day
0,2000-01,2000-08-19,H,H,Rob Harris,Charlton,Man City,0.666667,0.333333,11.000000,...,1.400000,0.400000,9.300000,5.200000,4.800000,10.400000,1.200000,0.200000,1.100000,6
1,2000-01,2000-08-19,H,H,Graham Barber,Chelsea,West Ham,1.000000,0.166667,7.833333,...,2.400000,1.100000,13.400000,6.300000,5.700000,12.800000,1.700000,0.000000,2.000000,6
2,2000-01,2000-08-19,A,D,Barry Knight,Coventry,Middlesbrough,2.500000,1.166667,13.000000,...,0.900000,0.700000,9.700000,3.200000,5.300000,16.200000,2.900000,0.100000,0.500000,6
3,2000-01,2000-08-19,D,A,Andy D'Urso,Derby,Southampton,1.500000,0.666667,8.666667,...,1.100000,0.400000,7.900000,4.500000,5.100000,14.100000,1.600000,0.100000,0.400000,6
4,2000-01,2000-08-19,H,H,Dermot Gallagher,Leeds,Everton,0.833333,0.000000,8.000000,...,1.700000,0.600000,11.800000,4.700000,6.400000,12.900000,1.700000,0.000000,1.300000,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8284,2021-22,2022-04-09,A,A,A Marriner,Watford,Leeds,0.600000,0.300000,11.000000,...,0.571429,0.428571,10.428571,3.285714,3.714286,10.714286,1.142857,0.142857,0.000000,6
8285,2021-22,2022-04-10,H,D,M Atkinson,Brentford,West Ham,1.100000,0.400000,11.400000,...,1.142857,0.142857,11.714286,3.857143,4.285714,8.142857,1.285714,0.142857,1.428571,7
8286,2021-22,2022-04-10,H,H,R Jones,Leicester,Crystal Palace,1.400000,0.700000,10.300000,...,1.571429,0.857143,11.285714,3.285714,3.285714,8.571429,0.714286,0.000000,2.000000,7
8287,2021-22,2022-04-10,D,H,A Taylor,Man City,Liverpool,2.800000,1.500000,15.500000,...,3.428571,1.857143,18.571429,8.000000,8.714286,8.285714,0.428571,0.000000,2.285714,7


In [10]:
data.select_dtypes('number').sum().sum()-data['day'].sum()
# data.shape

1311092.7055555554